In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

In [2]:
if torch.backends.mps.is_available():
    print("MPS có sẵn.")
    device = torch.device("mps")
else:
    print("MPS không có sẵn. Đang sử dụng CPU.")
    device = torch.device("cpu")

print(f"Thiết bị đang sử dụng: {device}")

# Ví dụ chuyển tensor lên GPU
x = torch.ones(5, device=device)
print(f"Tensor đã được tạo trên thiết bị: {x.device}")

MPS có sẵn.
Thiết bị đang sử dụng: mps
Tensor đã được tạo trên thiết bị: mps:0


In [ ]:
class DoubleConv(nn.Module):
    def __int__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv  = nn.Sequential(
            # Mục tiêu: học các đặc trưng cơ bản (edges, contours, v.v.) từ ảnh đầu vào, sau đó kết hợp chúng để học các đặc trưng phức tạp hơn.
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            # Bước này không thay đổi số kênh, chỉ giúp mô hình học sâu hơn trên cùng độ phân giải.
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self    , x):
        return self.conv(x)
class Unet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(Unet, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Phần giảm kích thước (Encoder)
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Phần tăng kích thước (Decoder)
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2)
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)  # ConvTranspose2d
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)  # Nối theo kênh
            x = self.ups[idx+1](concat_skip)  # DoubleConv

        return self.final_conv(x)